In [41]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import webbrowser
import pyttsx3
import json
import threading
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
import numpy as np
import speech_recognition as sr
import math
import random
from flask import Flask, request, jsonify

In [42]:
try:
    with open(r"D:\College\jarvis\intents.json", "r") as f:
        intent_data = json.load(f)
except Exception as e:
    print(" Error loading intents.json:", e)
    intent_data = {
        "hello": "Hi there!",
        "bye": "Goodbye!",
        "tell me a joke": [
            "Why don't scientists trust atoms? Because they make up everything!",
            "Why did the scarecrow win an award? Because he was outstanding in his field!"
        ]
    }

In [43]:
phrases = list(intent_data.keys())
responses_map = intent_data
labels = list(intent_data.keys())

model = SentenceTransformer('all-MiniLM-L6-v2')
X_embeddings = model.encode(phrases)

le = LabelEncoder()
y_encoded = le.fit_transform(labels)

clf = LogisticRegression()
clf.fit(X_embeddings, y_encoded)

d:\College\jarvis\.venv\Lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  y_type = type_of_target(y, input_name="y")


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [44]:
def speak(text):
    def speak_thread():
        try:
            engine = pyttsx3.init()
            engine.say(text)
            engine.runAndWait()
        except Exception as e:
            print("Speech error:", e)
    threading.Thread(target=speak_thread, daemon=True).start()

def listen_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        speak("Listening...")
        print("Listening...")
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=8)
            user_input = recognizer.recognize_google(audio)
            return user_input
        except sr.UnknownValueError:
            return "Sorry, I didn't catch that."
        except sr.RequestError:
            return "Voice service is unavailable."
        except sr.WaitTimeoutError:
            return "You didn't say anything."

In [45]:
def get_response(user_input):
    global current_city

    lower_input = user_input.lower().strip()
    time_phrases = ["what time is it", "whats the time", "current time", "tell me the time"]
    weather_phrases = ["weather", "whats the weather", "weather update", "weather today"]

    if lower_input in time_phrases:
        return datetime.now().strftime("It's %I:%M %p right now.")

    if any(phrase in lower_input for phrase in weather_phrases):
        return fetch_weather(current_city)

    if lower_input.startswith("set city to"):
        new_city = lower_input.replace("set city to", "").strip()
        if new_city:
            current_city = new_city.title()
            return f"Got it! I'll now give weather updates for {current_city}."
        else:
            return "Please specify a valid city name."

    if lower_input.startswith("play on youtube"):
        query = lower_input.replace("play on youtube", "").strip()
        if query:
            search_url = f"https://www.youtube.com/results?search_query={query.replace(' ', '+')}"
            webbrowser.open(search_url)
            return f"Searching and playing '{query}' on YouTube..."
        else:
            return "Please tell me what to play on YouTube."

    if lower_input.startswith("play on spotify"):
        query = lower_input.replace("play on spotify", "").strip()
        if query:
            search_url = f"https://open.spotify.com/search/{query.replace(' ', '%20')}"
            webbrowser.open(search_url)
            return f"Searching '{query}' on Spotify..."
        else:
            return "Please tell me what to search on Spotify."

In [46]:
def get_response(user_input):
    global current_city

    lower_input = user_input.lower().strip()
    time_phrases = ["what time is it", "whats the time", "current time", "tell me the time"]
    weather_phrases = ["weather", "whats the weather", "weather update", "weather today"]

    if lower_input in time_phrases:
        return datetime.now().strftime("It's %I:%M %p right now.")

    if lower_input.startswith("set city to"):
        new_city = lower_input.replace("set city to", "").strip()
        if new_city:
            current_city = new_city.title()
            return f"Got it! I'll now give weather updates for {current_city}."
        else:
            return "Please specify a valid city name."

    if lower_input.startswith("play on youtube"):
        query = lower_input.replace("play on youtube", "").strip()
        if query:
            search_url = f"https://www.youtube.com/results?search_query={query.replace(' ', '+')}"
            webbrowser.open(search_url)
            return f"Searching and playing '{query}' on YouTube..."
        else:
            return "Please tell me what to play on YouTube."

    if lower_input.startswith("play on spotify"):
        query = lower_input.replace("play on spotify", "").strip()
        if query:
            search_url = f"https://open.spotify.com/search/{query.replace(' ', '%20')}"
            webbrowser.open(search_url)
            return f"Searching '{query}' on Spotify..."
        else:
            return "Please tell me what to search on Spotify."

In [47]:
class JarvisUI:
    def __init__(self, root):
        self.root = root
        self.setup_window()
        self.setup_styles()
        self.create_widgets()

    def setup_window(self):
        self.root.title("J.A.R.V.I.S - Personal Assistant")
        self.root.geometry("800x600")
        self.root.configure(bg='#0a0a0a')
        self.root.resizable(True, True)
        x = (self.root.winfo_screenwidth() // 2) - (800 // 2)
        y = (self.root.winfo_screenheight() // 2) - (600 // 2)
        self.root.geometry(f'800x600+{x}+{y}')

    def setup_styles(self):
        style = ttk.Style()
        style.theme_use('clam')
        style.configure('Title.TLabel', background='#0a0a0a', foreground='#00d4ff', font=('Helvetica', 20, 'bold'))
        style.configure('Status.TLabel', background='#0a0a0a', foreground='#00ff41', font=('Helvetica', 10))
        style.configure('Custom.TButton', background='#1a1a1a', foreground='#00d4ff', font=('Helvetica', 12, 'bold'))
        style.map('Custom.TButton', background=[('active', '#2a2a2a'), ('pressed', '#3a3a3a')])

    def create_widgets(self):
        main_frame = tk.Frame(self.root, bg='#0a0a0a')
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)

        header_frame = tk.Frame(main_frame, bg='#0a0a0a')
        header_frame.pack(fill='x', pady=(0, 20))

        title_label = ttk.Label(header_frame, text="J.A.R.V.I.S", style='Title.TLabel')
        title_label.pack(side='left')

        self.status_label = ttk.Label(header_frame, text="● Online", style='Status.TLabel')
        self.status_label.pack(side='right')

        chat_frame = tk.Frame(main_frame, bg='#1a1a1a', relief='sunken', bd=2)
        chat_frame.pack(fill='both', expand=True, pady=(0, 20))

        self.chat = scrolledtext.ScrolledText(
            chat_frame, bg='#0a0a0a', fg='#00ff41', font=('Consolas', 11), wrap=tk.WORD,
            state='disabled', selectbackground='#2a2a2a', selectforeground='#00d4ff',
            insertbackground='#00ff41', bd=0, padx=15, pady=10
        )
        self.chat.pack(fill='both', expand=True, padx=2, pady=2)

        self.chat.tag_config('user', foreground='#00d4ff', font=('Consolas', 11, 'bold'))
        self.chat.tag_config('jarvis', foreground='#00ff41', font=('Consolas', 11))
        self.chat.tag_config('timestamp', foreground='#666666', font=('Consolas', 9))

        input_frame = tk.Frame(main_frame, bg='#0a0a0a')
        input_frame.pack(fill='x')

        self.entry = tk.Entry(
            input_frame, bg='#1a1a1a', fg='#00d4ff', font=('Helvetica', 12),
            insertbackground='#00d4ff', bd=2, relief='sunken',
            selectbackground='#2a2a2a', selectforeground='#00ff41'
        )
        self.entry.pack(side='left', fill='x', expand=True, padx=(0, 10))
        self.entry.bind('<Return>', lambda event: self.on_submit())

        self.submit_btn = ttk.Button(input_frame, text="Send", command=self.on_submit, style='Custom.TButton')
        self.submit_btn.pack(side='right', padx=(0, 10))

        self.voice_btn = ttk.Button(input_frame, text="🎤 Voice", command=self.handle_voice_input, style='Custom.TButton')
        self.voice_btn.pack(side='right', padx=(0, 10))

        self.clear_btn = ttk.Button(input_frame, text="Clear", command=self.clear_chat, style='Custom.TButton')
        self.clear_btn.pack(side='right')

        self.add_message("jarvis", "Hello! I'm J.A.R.V.I.S, your personal assistant. How can I help you today?", show_time=True)
        self.entry.focus()

    def add_message(self, sender, message, show_time=False):
        self.chat.config(state='normal')
        if show_time:
            timestamp = datetime.now().strftime("%H:%M")
            self.chat.insert(tk.END, f"[{timestamp}] ", 'timestamp')

        if sender == "user":
            self.chat.insert(tk.END, "You: ", 'user')
        else:
            self.chat.insert(tk.END, "J.A.R.V.I.S: ", 'jarvis')

        self.chat.insert(tk.END, f"{message}\n\n", sender)
        self.chat.config(state='disabled')
        self.chat.see(tk.END)

    def on_submit(self):
        user_input = self.entry.get().strip()
        if not user_input:
            return

        self.entry.delete(0, tk.END)
        self.add_message("user", user_input, show_time=True)
        self.status_label.config(text="● Processing...")
        self.root.update()

        try:
            response = get_response(user_input)
            self.add_message("jarvis", response, show_time=True)
            speak(response)
        except Exception as e:
            self.add_message("jarvis", f"Sorry, I encountered an error: {str(e)}", show_time=True)

        self.status_label.config(text="● Online")
        self.entry.focus()

    def handle_voice_input(self):

        voice_text = listen_voice_input()
        self.add_message("user", voice_text, show_time=True)

        try:
            response = get_response(voice_text)
            self.add_message("jarvis", response, show_time=True)
            speak(response)
        except Exception as e:
            self.add_message("jarvis", f"Sorry, I encountered an error: {str(e)}", show_time=True)

        self.status_label.config(text="● Online")

    def clear_chat(self):
        self.chat.config(state='normal')
        self.chat.delete(1.0, tk.END)
        self.chat.config(state='disabled')
        self.add_message("jarvis", "Chat cleared. How can I help you?", show_time=True)

In [48]:
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = JarvisUI(root)
        root.mainloop()
    except Exception as e:
        print("Startup Error:", e)
